In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# load ascii text and covert to lowercase
filename = "yoda.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [3]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  8911
Total Vocab:  45


In [5]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  8811


In [6]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [7]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# define the checkpoint
filepath="yoda-nn-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

In [8]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [10]:
import sys
# load the network weights
filename = "yoda-nn-114-0.1921.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("---" * 10)
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
print("---" * 10)
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("---" * 10)
print ("\nDone.")

------------------------------
Seed:
" e your feelings, obi-wan, and find him you will.
already know you that which you need.
adventure. ex "
------------------------------
citement. a jedi craves not these things. hes job it is.
the dark side clouds everything. impossible to see the fithrh ine dist siee cl the eadt sadth teus. forind woir hs waet.oo whi cetc s cereds isdie  amamut them tou atoul oo she force  rou d eeleno in dade.t atrued toan ssods oo toen.  foaal ysu stint not  foo yhe lrrc sare l ootereds 
lrk  alwees wou  hom the dark sitt,llousthave,tho oo jeaigo fo nen hitnee woat in woe yot hhfe disd on tne forczes. anme s ltle. to thes  so mwle foo seeer to pesgel yo lhe gosdo dod .i aod  offe  rer rorrete  fos so ted fotgt 
ithe:errcse ane you axow dha dor en co dor en ac.drehere  aere i meee mo wee gorues shel  wou miet you bile diet on al t.c force are thes. aasily they flow, quick to join you in a fight. if once you start down the dark path, forever will it dominate your d

In [ ]:
pattern